In [135]:
import numpy as np 
import pandas as pd 
import json
from pandas.io.json import json_normalize
import os
from sklearn.model_selection import StratifiedKFold
import torch
import random
from torch.nn import Linear, LayerNorm, ReLU, Dropout
from torch_geometric.nn import ChebConv, NNConv, DeepGCNLayer
from torch_geometric.data import Data, DataLoader
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [2]:
train = pd.read_json('../train.json', lines=True)

In [3]:
train=train.set_index("index")

In [4]:
train=train[train["signal_to_noise"]>=1]

In [5]:
train=train.assign(pair_prob=[np.load('../bpps/'+row['id']+'.npy') for index,row in train.iterrows()]) 

In [31]:
bpps_nb_mean = 0.077522 # mean of bpps_nb across all training data
bpps_nb_std = 0.08914  
error_mean_limit = 0.5

In [92]:
target_col = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

rna_dict    = {x:i for i, x in enumerate('ACGU')} #4
struct_dict = {x:i for i, x in enumerate('().')}  #3
loop_dict   = {x:i for i, x in enumerate('BEHIMSX')}#7

In [108]:
pair_threshold = 0.24
def create_adj(index):
    mat = np.array(train.iloc[index]["pair_prob"])
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if(i==j):
                mat[i][j] = 1
                continue
            if(mat[i][j]>pair_threshold):
                mat[i][j]=1
            else:
                mat[i][j]=0
    return mat
def calc_error_mean(row):
    reactivity_error = row['reactivity_error']
    deg_error_Mg_pH10 = row['deg_error_Mg_pH10']
    deg_error_Mg_50C = row['deg_error_Mg_50C']

    return np.mean(np.abs(reactivity_error) +
                   np.abs(deg_error_Mg_pH10) + \
                   np.abs(deg_error_Mg_50C)) / 3

def calc_sample_weight(row):

    error_mean = calc_error_mean(row)
    if error_mean >= error_mean_limit:
        return 0.

    return 1. - error_mean / error_mean_limit
def weighted_mse_loss(prds, tgts, weight):
    return torch.mean(weight * (prds - tgts)**2)

def criterion(prds, tgts, weight=None):
    
    return (torch.sqrt(torch.nn.MSELoss()(prds[:,0], tgts[:,0])) +
            torch.sqrt(torch.nn.MSELoss()(prds[:,1], tgts[:,1])) +
            torch.sqrt(torch.nn.MSELoss()(prds[:,2], tgts[:,2]))) / 3
    
def add_edges(edge_index, edge_features, node1, node2, feature1, feature2):
    edge_index.append([node1, node2])
    edge_features.append(feature1)
    edge_index.append([node2, node1])
    edge_features.append(feature2)

def add_edges_between_base_nodes(edge_index, edge_features, node1, node2):
    edge_feature1 = [
        0, # is edge for paired nodes
        0, # is edge between codon node and base node
        0, # is edge between coden nodes
        1, # forward edge: 1, backward edge: -1
        1, # bpps if edge is for paired nodes
    ]
    edge_feature2 = [
        0, # is edge for paired nodes
        0, # is edge between codon node and base node
        0, # is edge between coden nodes
        -1, # forward edge: 1, backward edge: -1
        1, # bpps if edge is for paired nodes
    ]
    add_edges(edge_index, edge_features, node1, node2,
              edge_feature1, edge_feature2)

def add_edges_between_paired_nodes(edge_index, edge_features, node1, node2,
                                   bpps_value):
    edge_feature1 = [
        1, # is edge for paired nodes
        0, # is edge between codon node and base node
        0, # is edge between coden nodes
        0, # forward edge: 1, backward edge: -1
        bpps_value, # bpps if edge is for paired nodes
    ]
    edge_feature2 = [
        1, # is edge for paired nodes
        0, # is edge between codon node and base node
        0, # is edge between coden nodes
        0, # forward edge: 1, backward edge: -1
        bpps_value, # bpps if edge is for paired nodes
    ]
    add_edges(edge_index, edge_features, node1, node2,
              edge_feature1, edge_feature2)

def add_edges_between_codon_nodes(edge_index, edge_features, node1, node2):
    edge_feature1 = [
        0, # is edge for paired nodes
        0, # is edge between codon node and base node
        1, # is edge between coden nodes
        1, # forward edge: 1, backward edge: -1
        0, # bpps if edge is for paired nodes
    ]
    edge_feature2 = [
        0, # is edge for paired nodes
        0, # is edge between codon node and base node
        1, # is edge between coden nodes
        -1, # forward edge: 1, backward edge: -1
        0, # bpps if edge is for paired nodes
    ]
    add_edges(edge_index, edge_features, node1, node2,
              edge_feature1, edge_feature2)

def add_edges_between_codon_and_base_node(edge_index, edge_features,
                                          node1, node2):
    edge_feature1 = [
        0, # is edge for paired nodes
        1, # is edge between codon node and base node
        0, # is edge between coden nodes
        0, # forward edge: 1, backward edge: -1
        0, # bpps if edge is for paired nodes
    ]
    edge_feature2 = [
        0, # is edge for paired nodes
        1, # is edge between codon node and base node
        0, # is edge between coden nodes
        0, # forward edge: 1, backward edge: -1
        0, # bpps if edge is for paired nodes
    ]
    add_edges(edge_index, edge_features, node1, node2,
              edge_feature1, edge_feature2)

def add_node(node_features, feature):
    node_features.append(feature)

def add_base_node(node_features, sequence, predicted_loop_type,
                  bpps_sum, bpps_nb):
    feature = [
        0, # is codon node
        sequence == 'A',
        sequence == 'C',
        sequence == 'G',
        sequence == 'U',
        predicted_loop_type == 'S',
        predicted_loop_type == 'M',
        predicted_loop_type == 'I',
        predicted_loop_type == 'B',
        predicted_loop_type == 'H',
        predicted_loop_type == 'E',
        predicted_loop_type == 'X',
        bpps_sum,
        bpps_nb,
    ]
    add_node(node_features, feature)

def add_codon_node(node_features):
    feature = [
        1, # is codon node
        0, # sequence == 'A',
        0, # sequence == 'C',
        0, # sequence == 'G',
        0, # sequence == 'U',
        0, # predicted_loop_type == 'S',
        0, # predicted_loop_type == 'M',
        0, # predicted_loop_type == 'I',
        0, # predicted_loop_type == 'B',
        0, # predicted_loop_type == 'H',
        0, # predicted_loop_type == 'E',
        0, # predicted_loop_type == 'X',
        0, # bpps_sum
        0, # bpps_nb
    ]
    add_node(node_features, feature)

    
def build_data(df, is_train):
    data = []
    for i in range(len(df)):
        targets = []
        node_features = []
        edge_features = []
        edge_index = []
        train_mask = []
        test_mask = []
        weights = []

        id = df.loc[i, 'id']
        path = os.path.join('../bpps', id + '.npy')
        bpps = np.load(path)
        bpps_sum = bpps.sum(axis=0)
        sequence = df.loc[i, 'sequence']
        structure = df.loc[i, 'structure']
        pair_info = match_pair(structure)
        predicted_loop_type = df.loc[i, 'predicted_loop_type']
        seq_length = df.loc[i, 'seq_length']
        seq_scored = df.loc[i, 'seq_scored']
        bpps_nb = (bpps > 0).sum(axis=0) / seq_length
        bpps_nb = (bpps_nb - bpps_nb_mean) / bpps_nb_std
        if is_train:
            sample_weight = calc_sample_weight(df.loc[i])

            reactivity = df.loc[i, 'reactivity']
            deg_Mg_pH10 = df.loc[i, 'deg_Mg_pH10']
            deg_Mg_50C = df.loc[i, 'deg_Mg_50C']

            for j in range(seq_length):
                if j < seq_scored:
                    targets.append([
                        reactivity[j],
                        deg_Mg_pH10[j],
                        deg_Mg_50C[j],
                        ])
                else:
                    targets.append([0, 0, 0])

        paired_nodes = {}
        for j in range(seq_length):
            add_base_node(node_features, sequence[j], predicted_loop_type[j],
                          bpps_sum[j], bpps_nb[j])

            if j + 1 < seq_length: # edge between current node and next node
                add_edges_between_base_nodes(edge_index, edge_features,
                                             j, j + 1)

            if pair_info[j] != -1:
                if pair_info[j] not in paired_nodes:
                    paired_nodes[pair_info[j]] = [j]
                else:
                    paired_nodes[pair_info[j]].append(j)

            train_mask.append(j < seq_scored)
            test_mask.append(True)
            if is_train:
                weights.append(sample_weight)

       
        for pair in paired_nodes.values():
            bpps_value = bpps[pair[0], pair[1]]
            add_edges_between_paired_nodes(edge_index, edge_features,
                                           pair[0], pair[1], bpps_value)

       
        codon_node_idx = seq_length - 1
        for j in range(seq_length):
            if j % 3 == 0:
                # add codon node
                add_codon_node(node_features)
                codon_node_idx += 1
                train_mask.append(False)
                test_mask.append(False)
                if is_train:
                    weights.append(0)
                    targets.append([0, 0, 0])

                if codon_node_idx > seq_length:
                    # add edges between adjacent codon nodes
                    add_edges_between_codon_nodes(edge_index, edge_features,
                                                  codon_node_idx - 1,
                                                  codon_node_idx)

            # add edges between codon node and base node
            add_edges_between_codon_and_base_node(edge_index, edge_features,
                                                  j, codon_node_idx)

        node_features = torch.tensor(node_features, dtype=torch.float)
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_features = torch.tensor(edge_features, dtype=torch.float)

        if is_train:
            data.append(MyData(x=node_features, edge_index=edge_index,
                               edge_attr=edge_features,
                               train_mask=torch.tensor(train_mask),
                               weight=torch.tensor(weights, dtype=torch.float),
                               y=torch.tensor(targets, dtype=torch.float)))
        else:
            data.append(MyData(x=node_features, edge_index=edge_index,
                               edge_attr=edge_features,
                               test_mask=torch.tensor(test_mask)))

    return data

def np_onehot(x, max=54):
    return np.eye(max)[x]

In [263]:
class CNN2D(nn.Module):
    def __init__(self):
        super(CNN2D, self).__init__()
        self.conv1 = nn.Conv2d(2, 4, 2)
        self.conv2 = nn.Conv2d(4, 10, 1)
        self.fc1 = nn.Linear(10 * 33, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 68)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        
        x = F.relu(self.conv2(x))
        x = x.view(-1, 10 * 33)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x


net = CNN2D()

In [46]:
if __name__ == '__main__':
    seed = 777
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    kf = StratifiedKFold(5, shuffle=True, random_state=seed)
    

In [247]:
sequence=train['sequence'].map(lambda seq: [rna_dict[x] for x in seq]).tolist()
struct = train['structure'].map(lambda seq: [struct_dict[x] for x in seq]).tolist()
loop   = train['predicted_loop_type'].map(lambda seq: [loop_dict[x] for x in seq]).tolist()

In [122]:
def get_train_item(index):
    data = []
    data.append(np.array(np_onehot(sequence[index],4)))
    data.append(np_onehot(loop[index],7))
    data.append(create_adj(index))
    return (data)

In [219]:
def get_folded_seq(index):
    final = []
    seq = sequence[index]
    if(len(seq)==107):
        seq = seq[0:68]
        final.append(seq[0:34])
        final.append(seq[34:68])
    else:
        seq = seq[0:92]
        final.append(seq[0:34])
        final.append(seq[34:68])
    return final

def get_folded_loop(index):
    final = []
    seq = loop[index]
    if(len(seq)==107):
        seq = seq[0:68]
        final.append(seq[0:34])
        final.append(seq[34:68])
    else:
        seq = seq[0:92]
        final.append(seq[0:34])
        final.append(seq[34:68])
    return final

def build_channel(index):    
    final = []
    for x,y in zip(get_folded_seq(index),get_folded_loop(index)):
        tmp = []
        for a,b in zip(x,y):
            tmp.append([a,b])
        final.append(tmp)
    return [final]

In [265]:
for x in range(len(train)):
    t=torch.Tensor(build_channel(x))
    print(np.array(net(t).tolist()[0])-np.array(train.iloc[x]["reactivity"]))

[-0.16678143 -1.52507189 -1.10817476 -0.78759294 -0.77093128 -0.41783431
 -0.13803642 -0.19419552 -0.22202398 -0.40143637 -1.0017422  -1.52122102
 -1.6173147  -1.18248093 -1.02266873 -0.5962496  -0.36735436 -0.13441771
 -0.05840697 -0.31737337 -0.13635672 -0.77624665 -0.8588762  -0.20620439
 -0.31330452 -0.04251441 -0.24344048 -0.82670903 -0.29849363 -0.34203905
 -0.93997701 -0.5157836  -0.73112367 -1.00339479 -0.73417915 -1.05026178
 -0.94491118 -0.47520101 -0.18829449 -0.0429614  -0.09731304 -0.07078468
 -0.29658155  0.01321462 -0.52634877 -1.11928145 -0.34502594 -0.22307504
  0.09155327 -0.14071992 -0.24143842 -0.43744299 -0.96852745 -0.48408256
 -0.59830043 -0.22861364 -0.261798   -0.26795004 -0.26367172 -0.40266855
 -0.62038671 -0.4234373  -0.09240769 -0.19104013 -0.11774724 -0.24839765
 -0.21287451 -0.51746627]
[-0.43028049 -1.48442169 -1.15775041 -0.72072471 -0.7521014  -0.61572136
 -0.08468742 -0.01911268 -0.29486386 -0.38837948 -0.56387478 -0.37309847
 -1.00441706 -0.59659478 

 -0.05487054 -0.12755204]
[-0.31928842 -2.16395845 -1.15599643 -0.64752745 -0.53730964  0.02989893
  0.14001552  0.09765044 -0.15341642 -0.07144266 -0.05279973 -0.08218114
  0.09157056  0.03906269 -0.01769642  0.07873222  0.04965096 -0.26667571
 -1.21316603 -0.47754979 -1.84397397 -0.18023019 -0.14256784 -0.04084225
 -0.17066598 -0.06797017 -0.13909697 -0.87802603 -2.29185406 -0.44901081
  0.10015758 -0.1531707  -0.18488961 -0.76127738 -1.72235207 -0.38764026
 -1.52934491 -1.36614877 -0.81319269 -1.50318912 -0.00839724  0.00303544
 -0.21822993 -0.06682022 -0.02509784 -0.09757991 -0.03795072 -0.08989391
  0.11264765 -0.16609396 -0.14795937 -0.2578914  -0.1116229   0.08785947
  0.05782361  0.01974329 -0.18386634 -0.15163227 -0.096879    0.06697154
 -0.05039284  0.03386483 -0.07856625 -0.14962409 -0.09981208 -0.72796949
 -0.7310626  -0.16346011]
[-0.45994643 -1.36459359 -1.0676208  -0.95938313 -0.43635126 -0.04788263
 -0.49872208 -0.66277354 -0.58566633 -0.71816367 -0.42273057 -0.32266907

 -1.12629121  0.03164272]
[-0.29596507 -0.50838868 -1.2072073  -1.04222698 -0.10296667 -0.08973298
 -0.07322664 -0.10732659 -1.00576694 -1.37333926 -0.66450917 -1.55832102
 -0.54665445 -0.05307226 -0.41440348 -0.47893071 -0.29546621 -0.23960093
 -0.15036024 -0.26932305 -0.24931235  0.00273937 -0.50850211 -0.80854923
 -0.04838844 -0.68539545 -1.16142028 -0.19326498 -0.06081899 -0.28477038
 -0.15033817  0.08108007 -0.99330576 -0.61594957  0.11931876 -0.18127351
 -0.42457448 -0.82736909 -0.81688982 -0.43846168 -0.39551938 -0.13273666
 -0.16572868 -0.37980525 -0.35744603 -0.24365206 -0.22884141 -0.80474412
 -0.2165419  -0.31005189 -1.00900649 -0.69553757 -0.63071137 -0.84467924
 -0.76635464 -0.06646433 -0.7996065  -1.24669325 -0.98943366 -0.1516961
 -0.53674687 -0.70652527 -0.53437166 -0.26033259 -0.70401129 -0.03105301
 -0.04528559 -0.00959849]
[-9.59814001e-01 -1.81170141e+00 -8.43227255e-01 -6.90422421e-01
 -1.66877399e-01 -2.11183091e-01  1.07134256e-01  1.72504456e-01
 -3.36073339e-02

 -0.2127558  -0.1535734 ]
[-0.48811182 -2.22595431 -1.54193449 -0.73560648 -0.58469699 -0.33515595
  0.14152767  0.22099751 -0.02673966  0.04988127 -0.14817638 -0.04759267
  0.06527823 -0.00292111 -0.74743015 -0.91415304 -0.22319595 -0.65311905
 -0.18972278 -0.24213777 -0.12298204  0.01441224 -0.33367353 -0.52216311
 -0.23100935 -0.62853187 -1.24969646 -0.31496246 -0.52052855 -0.20383305
 -0.69374921 -0.0267499  -0.19528052 -0.16008541  0.24216077 -0.08996562
  0.12156107 -0.04401486  0.13468574 -0.15385214 -1.46092827 -0.46375656
 -0.25032078  0.08356169 -0.72768182 -0.72766542 -1.13023201 -0.92030803
 -0.13758717 -0.16333935 -0.2933654   0.00447715  0.07028333  0.09157534
  0.10609532  0.08747698 -0.12906929 -0.15305918 -0.08793954  0.06764474
 -0.22589522 -0.32619099 -0.11132273 -0.35833753 -1.27987112 -0.90752998
 -0.68023633 -0.08663135]
[-0.33444643 -1.50099359 -1.0817208  -1.23388313 -0.23855126 -0.16938263
 -0.50362208 -0.80937354 -0.41586633 -0.69676367 -0.42943057 -0.42156907

  4.90348786e-03  8.36397111e-02 -2.36501490e-02 -3.05213489e-01]
[-2.71976422e-01 -1.98328421e+00 -9.95674024e-01 -1.16333230e+00
 -6.94109643e-01 -8.79501104e-03  7.45386939e-02  1.79948087e-01
 -1.16826892e-02 -1.46495002e-03 -3.28645987e-02 -7.03544079e-02
 -3.23484135e-02 -2.36525624e-01 -8.69246161e-01 -7.51919716e-01
 -3.00149461e-01 -1.14825617e-01  2.22616122e-02 -1.25456527e-01
 -1.24632061e-01 -7.81151117e-02 -2.51363827e-01 -5.06539437e-01
 -8.34451243e-02 -2.74675424e-01 -1.32892480e+00 -3.80073725e-01
 -1.69058725e+00 -1.35741855e+00 -4.89785330e-01 -1.21564269e-02
 -2.29467815e-01 -1.29396776e-01  2.18189184e-01 -2.33397109e-02
  1.29746556e-01 -3.46459523e-02  2.38467120e-02 -1.82236871e-01
 -1.38648275e+00 -2.98951753e-01 -1.67894952e-01  1.13801904e-01
 -6.67343686e-01 -8.32045046e-01 -5.30216670e-01 -7.67617692e-01
 -1.03440526e-01 -1.11440100e-01 -9.18589643e-02 -2.74736807e-02
  7.55432675e-02  1.24753643e-01  4.06073802e-02  1.35575802e-01
 -1.34971810e-01 -1.3250

...

 -0.00650751 -0.26779056]
[-0.27602073 -1.18111627 -1.22389612 -0.31231299 -0.19866707 -0.53582394
  0.12665872  0.09434042 -0.022948   -0.08437166 -0.18781332 -0.78515599
 -0.12130798  0.0305831  -0.00715453 -0.22239971 -0.54588465 -0.75242234
 -1.71269209 -0.60585264 -0.3979019  -0.0775054  -0.22103388 -0.34271396
 -0.08951572 -0.29600097 -0.51129735 -0.14010563 -0.30470993  0.10937984
 -0.14901556 -0.16427588 -0.40747799 -0.70942231  0.070382   -0.1772698
 -0.84566465 -0.34904653 -0.16896493 -0.41562533 -0.33696754 -0.61113734
 -1.65212597 -0.0565042  -0.05918134 -0.3843229  -0.09015039 -0.20703733
  0.06539858 -0.10553506 -0.18260334 -0.22362509 -0.41179478  0.03099217
 -0.75071931 -0.08208369 -0.20192221 -0.4454866  -0.22137875 -0.05605942
 -1.18727233 -0.38517007 -0.39172756 -1.61562427 -0.36359594 -0.10067198
 -0.1255271  -0.22550398]
[-0.58341359 -2.74827079 -0.46623799 -1.01421261 -1.79475658 -1.06510799
 -0.96023354 -1.66898587 -1.00483496 -1.27156789 -1.81726599 -0.55466404


 -0.5390948  -0.33910167]
[-0.63308116 -2.16966228 -1.27842598 -0.6230754  -0.6305722  -0.2029513
  0.05993034  0.14158946 -0.05258461 -0.07777507 -0.04177355 -1.14161641
 -0.04178994  0.04250675  0.04470904  0.03633008  0.05502102  0.03961567
 -0.05069347 -0.62726046 -0.79807957 -1.30390618 -0.70535954 -0.36524036
 -0.45693394 -0.02370385 -0.2982041  -0.66857987 -0.57633139 -0.0733342
 -0.06182848 -0.09782806 -0.20507946 -0.175467    0.00490077 -0.46970044
 -1.06244919 -0.19769064 -0.09138963 -0.08111743 -0.08595265 -0.06463121
 -0.1472913  -0.17347288 -0.41821786 -0.75240848 -0.44274716 -0.47201307
 -0.27064279 -0.32211198 -0.54512514  0.01845317 -0.04136102 -0.04002194
  0.10881666  0.11998769 -0.18789658 -0.15647149 -0.13550612 -0.015569
 -0.02153961 -0.64724478 -0.05114924 -0.15580593 -0.00433036  0.07370945
 -0.05959945 -0.04114052]
[-0.53024786 -1.32596806 -1.4093698  -0.47229285 -0.48782818 -0.13916717
 -0.09487242  0.11155208 -0.38851343 -0.35415219 -0.50948117 -0.46103244
 -1

 -0.0805575  -0.16729586]
[-0.49712302 -1.35698756 -1.03551779 -0.65198454 -0.28170389 -0.18525248
 -0.12968983  0.12796007 -0.08678743 -0.35169262 -1.49276998 -0.97445479
 -0.71368208 -0.16912815 -0.1503106  -0.54661722 -0.95971333 -0.90812778
 -0.27947162 -0.21342413 -0.21570147 -0.07238    -0.70659505 -1.60466937
 -1.08498311 -0.17863194 -0.74221857 -0.01641951 -0.34994296 -0.52042346
 -0.66538577 -0.24860504 -0.71254646 -0.37672225 -0.01178865 -0.458926
 -0.5824393  -0.08016362 -0.39415693 -0.76262709 -0.18520492 -0.72081297
 -0.96430679  0.05560307 -0.55903088 -1.2220543  -0.37992482 -0.7632829
 -0.92630925 -0.48341083 -0.3671667  -0.17226949 -0.12300802 -0.04029235
 -0.1665825   0.0710695  -0.19174078 -0.34650468 -0.12396039 -0.35156419
 -0.8473781  -0.39873404 -0.18978156 -0.94799362 -0.26032439 -0.51638726
 -0.30539473 -0.48646273]
[-0.47525126 -1.907297   -1.97791306 -0.95181834 -0.66925104 -0.13560947
  0.01357785  0.17272593 -0.11303711 -0.15222617 -0.57003407 -0.70149035
 -

 -6.14289587e-02  5.36101025e-02 -6.87639513e-03 -1.25568619e-02]
[ 0.06125271 -0.86725925 -0.704276   -0.91168952 -0.3619099  -0.16734197
 -0.34505727 -0.64623778 -0.21639208 -0.15228317 -1.06770305 -0.55365362
 -0.0992313  -0.57781441 -0.81833015 -0.12484717 -0.12969102 -0.47938848
 -0.27019874 -0.12498237 -0.08221741 -0.15831821 -0.15957312 -0.29942174
 -0.56646592 -0.22153952 -0.18860458 -0.23771524 -0.69774599 -0.32456341
 -0.07649223 -0.48228111 -0.75895545 -0.31803613  0.110236   -0.4887995
 -0.33741912 -0.04321384 -0.02214492 -0.38009117 -0.54962131 -0.58064572
 -0.22109132 -0.28521652 -0.29317999 -0.19228389 -0.04073252 -0.33244185
 -0.15369709 -0.08037374 -0.06371179 -0.14064237 -0.31848611 -0.06000567
  0.0175342  -0.24923858 -0.89280857 -0.55620331 -0.39406988 -0.57571799
 -0.63863958 -0.2237916  -0.2182971  -1.24648991 -0.86640557 -0.65384448
 -0.71227121 -0.35653053]
[-0.10834766 -0.77867488 -0.85608878 -0.45603919 -1.29002262 -0.87936379
 -0.06122317 -0.06814712 -0.21374

 -0.23250265 -0.52517939]
[-0.56354643 -1.40489359 -1.3087208  -1.18358313 -0.24865126 -0.01718263
 -0.51452208 -0.67947354 -0.56656633 -0.84136367 -0.46173057 -0.40026907
 -0.22844516 -0.91086418 -0.39614173 -0.1022244  -0.352425    0.05201073
 -0.41549389  0.30936805 -0.05692719  0.10020394  0.01237309 -0.31789256
 -0.23836925 -0.28183287 -0.29396556 -0.14720215 -0.23906615  0.20187398
 -0.07460588 -0.23168245 -0.35305274 -0.31325609  0.09564888 -0.14670898
  0.06840923 -0.06995784 -0.0872016  -0.05819818 -0.02766526 -0.11556868
 -0.20974866  0.03695242 -0.12421405 -0.19068519 -0.3633793  -0.30033004
 -0.35173112 -0.44723226 -0.23716607 -0.19985798 -0.70714277 -0.49325361
 -0.54986394 -0.09581574 -0.40093804 -0.97709139 -0.67030989 -0.45893392
 -0.64609775 -0.49948118 -0.85019548 -0.4030582  -0.14315363 -0.96381106
 -0.18759607 -0.0995866 ]
[-0.97443177 -1.80516978 -1.91472366 -0.88057391 -0.58241643 -0.18281617
  0.05373528  0.19643121 -0.09122849 -0.05457045 -0.32120272 -0.27994483

In [202]:
t.shape

torch.Size([2, 2, 34, 1])